# Math

We are going to implement all arithmetic operations here. These are the first opcodes that actually manipulate the stack and consume some gas.

Let's see how `add` works. We are going to `pop` 2 values from the stack add them and `push` back the result on the stack.

We also need to increment the `program counter (pc)` by one. In the end we need to deduct 3 gas for executing the `add` operation.

Let's declare some constants first. You will understand it as you read opcodes.

In [6]:
MOD  = 2**256         # 2**256
SIGN = 2**255
JUMPDEST=0x5B

Most arithmetic opcodes work like this. You can see in many opcodes, mode is used. The purpose is to convert it to uint256 before pushing to stack as stack can only take uint256 as per ethereum yellow paper.

In [7]:
def add(evm):
    a,b = evm.stack.pop(), evm.stack.pop()
    result = (a + b)%MOD
    evm.stack.push(result)
    evm.pc += 1
    evm.gas_dec(3)

In [8]:
def mul(evm):
    a,b = evm.stack.pop(), evm.stack.pop()
    result = (a * b) % MOD  
    evm.stack.push(result)
    evm.pc += 1
    evm.gas_dec(5)

In [9]:
def sub(evm):
    a,b = evm.stack.pop(), evm.stack.pop()
    result=(a-b)%MOD
    evm.stack.push(result)
    evm.pc += 1
    evm.gas_dec(3)    

One interesting note about how the EVM handles division by 0. Most other systems would throw an exception if you try to divide by 0. Not the EVM. It just returns 0.

Division by 0 are not directly handled by the EVM and are mostly a feature of the programming language like Solidity.

In [10]:
def div(evm):  #it will give correct result for unsigned integer only so always use it when dealing with unsigned number
    a,b = evm.stack.pop(), evm.stack.pop() #no need of modulo here because div cant overflow
    result = 0 if b == 0 else a // b
    evm.stack.push(result)
    evm.pc += 1
    evm.gas_dec(5)

sdiv is used when dealing with either or both of numerator and denominator being negative. We will first convert it to signed to calculate and the push it into stack converting to unsigned.

Two helper functions, one to convert unsigned to signed and vice-versa

In [11]:
def to_unsigned(x):
    return x % MOD

def to_signed(x):
    return x - MOD if x >= SIGN else x

In [12]:
def sdiv(evm):
    a,b= evm.stack.pop(), evm.stack.pop()
    a, b = to_signed(a), to_signed(b)   # reinterpret inputs as signed

    if b == 0:
        result = 0
    elif a == -(2**255) and b == -1:
        result = -(2**255)  #  case accroding to yellow paper
    else:
        result = int(a / b)   # truncates toward Zero in Python 

    evm.stack.push(to_unsigned(result))  # unsigned form
    evm.pc += 1
    evm.gas_dec(5)

In [13]:
def mod(evm):
    a, b = evm.stack.pop(), evm.stack.pop()
    evm.stack.push(0 if b == 0 else a % b)
    evm.pc += 1
    evm.gas_dec(5)

We need smod when either or both of numerator and denominator is negative unlike mod where both are positive. Here the procedure is similar to sdiv where it's first converted to signed, then we apply the logic and then push to stack converting again to unsigned.

In [14]:
def smod(evm):
    a,b = evm.stack.pop(), evm.stack.pop()
    a, b = to_signed(a), to_signed(b)  

    if b == 0:
        result = 0
    else:
        result = (abs(a) % abs(b)) * (1 if a >= 0 else -1)

    evm.stack.push(to_unsigned(result))
    evm.pc += 1
    evm.gas_dec(5)

In [15]:
def addmod(evm):
    a, b = evm.stack.pop(), evm.stack.pop()
    N = evm.stack.pop()
    result = 0 if N == 0 else (a + b) % N  #according to yellow paper 
    evm.stack.push(result)
    evm.pc += 1
    evm.gas_dec(8)

In [16]:
def mulmod(evm):
    a, b = evm.stack.pop(), evm.stack.pop()
    N = evm.stack.pop()
    result = 0 if N == 0 else (a * b) % N  #according to yellow paper 
    evm.stack.push(result)
    evm.pc += 1
    evm.gas_dec(8)

The gas cost for exp is dynamic. It is a function of how many bytes we need to represent the exponent in binary. This helper function calculates this.

In [17]:
def size_in_bytes(number):
    import math
    if number == 0: return 1
    bits_needed = math.ceil(math.log2(abs(number) + 1))
    return math.ceil(bits_needed / 8)

In [18]:
def exp(evm):
    a,exponent = evm.stack.pop(), evm.stack.pop()
    result = pow(a, exponent, MOD)   # safe and efficient  
    # result= (a**exponent)%MOD  same thing but less efficient
    evm.stack.push(result)
    evm.pc += 1
    evm.gas_dec(10 + (50 * size_in_bytes(exponent)))

More informations about this rarely used opcode `signextend` [here](https://ethereum.stackexchange.com/questions/63062/evm-signextend-opcode-explanation).

In [37]:
def signextend(evm):
    b, x = evm.stack.pop(), evm.stack.pop()
    if b <= 31:
        testbit = b * 8 + 7
        sign_bit = 1 << testbit
        if x & sign_bit: result = x | (2**256 - sign_bit)
        else           : result = x & (sign_bit - 1)
    else: result = x
    
    evm.stack.push(result)
    evm.pc += 1
    evm.gas_dec(5)